In [1]:
from cassandra_client import CassandraClient
import uuid
from datetime import datetime
import random
import threading

client = CassandraClient()

In [3]:
def stress_test_1(client, user_id, book_id, repetitions=10000):
    """The client makes the same request very quickly min (10000 times)."""
    reservation_ids = []
    for i in range(repetitions):
        reservation_id = client.make_reservation(user_id, book_id)
        if reservation_id:
            reservation_ids.append(reservation_id)
        #     print(f"Iteration {i+1}: Created reservation with ID {reservation_id}")
        # else:
        #     print(f"Iteration {i+1}: Failed to create reservation, book already reserved or does not exist.")
        if i % 100 == 0:
            print(f"Iteration {i+1}")
    print(f"Created {len(reservation_ids)} reservations in total")

client = CassandraClient()
client.delete_all_reservations()
user_id = uuid.uuid4()
book_id = uuid.UUID('79ec2f3d-8c09-4b7f-91ca-c7b8d0b27989')
stress_test_1(client, user_id, book_id)

KeyboardInterrupt: 

In [14]:
def stress_test_2(client, user_ids, book_ids, repetitions=10):
    """Two or more clients make the possible requests randomly (10000 times)."""
    def make_random_request():
        user_id = random.choice(user_ids)
        book_id = random.choice(book_ids)
        reservation_id = client.make_reservation(user_id, book_id)
        print(f"Created reservation for user {user_id} and book {book_id} with reservation ID {reservation_id}")

    threads = []
    for i in range(repetitions):
        t = threading.Thread(target=make_random_request)
        t.start()
        threads.append(t)
        print(f"Started thread {i+1}")

    for t in threads:
        t.join()

    print(f"Completed {repetitions} random reservations")

client.delete_all_reservations()
user_ids = [uuid.uuid4() for _ in range(5)]
book_ids = [uuid.uuid4() for _ in range(5)]
stress_test_2(client, user_ids, book_ids)

Started thread 1
Started thread 2
Started thread 3
Started thread 4
Started thread 5
Started thread 6
Started thread 7
Started thread 8
Started thread 9
Started thread 10
Created reservation for user aced1c2e-9faf-477a-9819-242b1f5e4d9e and book 3862d1c6-db92-4988-9db4-f46f10a96ae8 with reservation ID None
Created reservation for user b1617203-82be-4e4c-9018-09d62ea5236b and book 3862d1c6-db92-4988-9db4-f46f10a96ae8 with reservation ID None
Created reservation for user 5eddfa6a-2e74-4b2b-8bd1-fa35cc62153c and book 3dfba87a-972f-4381-973e-691e9c25f3b8 with reservation ID None
Created reservation for user aced1c2e-9faf-477a-9819-242b1f5e4d9e and book ca8d3f7f-7e51-4d15-9204-25c803c28316 with reservation ID None
Created reservation for user a86c7f07-9e2c-450f-b693-b21f7baca439 and book 3dfba87a-972f-4381-973e-691e9c25f3b8 with reservation ID None
Created reservation for user a86c7f07-9e2c-450f-b693-b21f7baca439 and book 3dfba87a-972f-4381-973e-691e9c25f3b8 with reservation ID None
Created

In [15]:
def stress_test_3(client, user_id_1, user_id_2, book_id, repetitions=10):
    """Immediate occupancy of all seats/reservations by 2 clients."""
    def make_reservations(user_id):
        for i in range(repetitions):
            reservation_id = client.make_reservation(user_id, book_id)
            print(f"User {user_id} made reservation {i+1} with ID {reservation_id}")

    t1 = threading.Thread(target=make_reservations, args=(user_id_1,))
    t2 = threading.Thread(target=make_reservations, args=(user_id_2,))

    t1.start()
    t2.start()
    print("Both threads started")

    t1.join()
    t2.join()
    print("Both threads completed")

client.delete_all_reservations()
user_id_1 = uuid.uuid4()
user_id_2 = uuid.uuid4()
book_id = uuid.uuid4()
stress_test_3(client, user_id_1, user_id_2, book_id)
print("All reservations deleted")

Both threads started
User 4b969566-6d20-4108-97b7-b280a48403a2 made reservation 1 with ID None
User 45692357-539f-410f-a93e-1cc848bd8664 made reservation 1 with ID None
User 4b969566-6d20-4108-97b7-b280a48403a2 made reservation 2 with ID None
User 45692357-539f-410f-a93e-1cc848bd8664 made reservation 2 with ID None
User 4b969566-6d20-4108-97b7-b280a48403a2 made reservation 3 with ID None
User 45692357-539f-410f-a93e-1cc848bd8664 made reservation 3 with ID None
User 45692357-539f-410f-a93e-1cc848bd8664 made reservation 4 with ID None
User 4b969566-6d20-4108-97b7-b280a48403a2 made reservation 4 with ID None
User 4b969566-6d20-4108-97b7-b280a48403a2 made reservation 5 with ID None
User 45692357-539f-410f-a93e-1cc848bd8664 made reservation 5 with ID None
User 45692357-539f-410f-a93e-1cc848bd8664 made reservation 6 with ID None
User 4b969566-6d20-4108-97b7-b280a48403a2 made reservation 6 with ID None
User 4b969566-6d20-4108-97b7-b280a48403a2 made reservation 7 with ID None
User 45692357-539

In [16]:
def stress_test_4(client, user_id, book_id, repetitions=10):
    """Constant cancellations and seat occupancy for the same seat (10000 times)."""
    def reserve_and_cancel():
        for i in range(repetitions):
            reservation_id = client.make_reservation(user_id, book_id)
            print(f"Iteration {i+1}: Created reservation with ID {reservation_id}")
            client.cancel_reservation(reservation_id)
            print(f"Iteration {i+1}: Cancelled reservation with ID {reservation_id}")

    t = threading.Thread(target=reserve_and_cancel)
    t.start()
    t.join()
    print(f"Completed {repetitions} reserve and cancel operations")

client.delete_all_reservations()
user_id = uuid.uuid4()
book_id = uuid.uuid4()
stress_test_4(client, user_id, book_id)

Exception in thread Thread-42 (reserve_and_cancel):
Traceback (most recent call last):
  File "c:\Users\Uni\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Uni\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Uni\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Uni\AppData\Local\Temp\ipykernel_19824\2151454665.py", line 7, in reserve_and_cancel
  File "c:\Users\Uni\Documents\Uni\sem6\BG\library-reservation\cassandra_client.py", line 53, in cancel_reservation
    self.session.execute(query, (reservation_id,))
  File "cassandra\cluster.py", line 2677, in cassandra.cluster.Session.execute
  File "cassandra\cluster.py", line 4956, in cassandra.cluster.ResponseFuture.result
cassandra.InvalidRequest: Error from server: code=2200 [Invali

Iteration 1: Created reservation with ID None
Completed 10 reserve and cancel operations


In [17]:
def stress_test_5(client, user_id, book_id, update_repetitions=10):
    """Update of 1000 reservations."""
    reservation_ids = [client.make_reservation(user_id, book_id) for _ in range(update_repetitions)]
    print(f"Created {len(reservation_ids)} reservations for update test")

    def update_reservations():
        new_date = datetime.now()
        for i, reservation_id in enumerate(reservation_ids):
            client.update_reservation(reservation_id, new_date)
            print(f"Updated reservation {i+1} with ID {reservation_id} to new date {new_date}")

    t = threading.Thread(target=update_reservations)
    t.start()
    t.join()
    print(f"Completed updates for {update_repetitions} reservations")

client.delete_all_reservations()
user_id = uuid.uuid4()
book_id = uuid.uuid4()
stress_test_5(client, user_id, book_id)

Exception in thread Thread-43 (update_reservations):
Traceback (most recent call last):
  File "c:\Users\Uni\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Uni\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Uni\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Uni\AppData\Local\Temp\ipykernel_19824\1239074434.py", line 9, in update_reservations
  File "c:\Users\Uni\Documents\Uni\sem6\BG\library-reservation\cassandra_client.py", line 40, in update_reservation
    self.session.execute(query, (new_date, reservation_id))
  File "cassandra\cluster.py", line 2677, in cassandra.cluster.Session.execute
  File "cassandra\cluster.py", line 4956, in cassandra.cluster.ResponseFuture.result
cassandra.InvalidRequest: Error from server: code=2

Created 10 reservations for update test
Completed updates for 10 reservations
